In [1]:
# Generate the data for slake, with the bbox annotations in slake
import json
ori_train_file = "/data/aofei/hallucination/Slake/train.json"
final_train_file = "/data/aofei/hallucination/Slake/data/training_gt_bbox.json"

In [2]:
with open(r"/data/aofei/hallucination/Slake/train.json", "r") as f:
    train_data = json.load(f)
len(train_data)

9835

In [3]:
train_data_en = []
for i in train_data:
    if i['q_lang'] == "en":
        train_data_en.append(i)
len(train_data_en)

4919

In [4]:
train_data_en[0]

{'img_id': 1,
 'img_name': 'xmlab1/source.jpg',
 'question': 'What modality is used to take this image?',
 'answer': 'MRI',
 'q_lang': 'en',
 'location': 'Abdomen',
 'modality': 'MRI',
 'answer_type': 'OPEN',
 'base_type': 'vqa',
 'content_type': 'Modality',
 'triple': ['vhead', '_', '_'],
 'qid': 0}

In [5]:
import os
image_name_segments_dict = dict()
for i in train_data_en:
    detections_path = os.path.join("/data/aofei/hallucination/Slake/imgs/" + i['img_name'].split("/")[0], "detection.json")
    with open(detections_path, "r") as f:
        bbox = json.load(f)
    image_name_segments_dict[i['img_name']] = bbox

In [6]:
image_name_segments_dict['xmlab1/source.jpg']

[{'Liver': [54.0, 106.0, 30.0, 31.0]}]

In [7]:
new_train_data = []
for i in train_data_en:
    template = dict()
    
    # template['answer_type'] = i['answer_type']
    template['image'] = i['img_name']
    template['id'] = i['qid']
    template['location'] = i['location']
    template['conversations'] = []
    # template['text'] = i['question']

    new_qa = {"from": "human", "value": "<image>\n" + i['question']}
    new_qa2 = {"from": "gpt", "value": str(i['answer'])}
    template['conversations'] += [new_qa, new_qa2]
    template['bboxes'] = []
    template['bboxes_dict'] = dict()
    for box_dict in image_name_segments_dict[i['img_name']]:
        k, bbox = list(box_dict.items())[0]
        if k.lower() in i['question'].lower():
            template['bboxes'].append(bbox)
            template['bboxes_dict'][k.lower()] = bbox
    new_train_data.append(template)

In [8]:
new_train_data[6]

{'image': 'xmlab1/source.jpg',
 'id': 6,
 'location': 'Abdomen',
 'conversations': [{'from': 'human', 'value': '<image>\nWhere is the liver?'},
  {'from': 'gpt', 'value': 'Right'}],
 'bboxes': [[54.0, 106.0, 30.0, 31.0]],
 'bboxes_dict': {'liver': [54.0, 106.0, 30.0, 31.0]}}

In [9]:
train_data_abd, train_data_lung = [], []
for i in new_train_data:
    if i['location'].lower() == "abdomen":
        train_data_abd.append(i)
    elif i['location'].lower() == "lung":
        train_data_lung.append(i)
len(train_data_lung), len(train_data_abd)

(1710, 1506)

In [10]:
with open(r"/data/aofei/hallucination/Slake/data/training_gt_bboxes_abd.json", "w") as f:
    json.dump(train_data_abd, f, indent=4)
with open(r"/data/aofei/hallucination/Slake/data/training_gt_bboxes_lung.json", "w") as f:
    json.dump(train_data_lung, f, indent=4)
with open(r"/data/aofei/hallucination/Slake/data/training_gt_bboxes.json", "w") as f:
    json.dump(new_train_data, f, indent=4)

In [11]:
s = 0
for i in train_data_lung:
    if len(i['bboxes']) > 0:
        s+=1
print(s / len(train_data_lung))

s = 0
for i in train_data_abd:
    if len(i['bboxes']) > 0:
        s+=1
print(s / len(train_data_abd))

0.07543859649122807
0.21713147410358566


In [12]:
import json
with open(r"/data/aofei/hallucination/Slake/data/training_gt_bboxes_abd.json", "r") as f:
    train_data_abd = json.load(f)
len(train_data_abd)

1506

In [13]:
bbox_data_abd = []
for i in train_data_abd:
    if len(i['bboxes']) != 0:
        bbox_data_abd.append(i)
len(bbox_data_abd)

327

In [5]:
with open(r"/data/aofei/hallucination/Slake/data/training_gt_only_bboxes_abd.json", "w") as f:
    json.dump(bbox_data_abd, f, indent=4)

In [7]:
bbox_data_abd[90]

{'image': 'xmlab230/source.jpg',
 'id': 1035,
 'location': 'Abdomen',
 'conversations': [{'from': 'human', 'value': '<image>\nWhere is the liver?'},
  {'from': 'gpt', 'value': 'Left'}],
 'bboxes': [[97.0, 186.0, 47.0, 59.0]]}